In [1]:
''' Getting all the clevage sites across the covid proteome and 
finding all the epitopes affected

Combine the glycosolation with the epistatic mutations and the cleavage. 
Will also have the protein and position inside of it
'''
import gzip
from Bio import SeqIO
from Bio.Seq import Seq
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
amino_acid_files = !ls data/v6/*_protein_*
amino_acid_files

['data/v6/aligned_protein_E.fasta',
 'data/v6/aligned_protein_M.fasta',
 'data/v6/aligned_protein_N.fasta',
 'data/v6/aligned_protein_ORF10.fasta',
 'data/v6/aligned_protein_ORF14.fasta',
 'data/v6/aligned_protein_ORF1a.fasta',
 'data/v6/aligned_protein_ORF1b.fasta',
 'data/v6/aligned_protein_ORF3a.fasta',
 'data/v6/aligned_protein_ORF6.fasta',
 'data/v6/aligned_protein_ORF7a.fasta',
 'data/v6/aligned_protein_ORF7b.fasta',
 'data/v6/aligned_protein_ORF8.fasta',
 'data/v6/aligned_protein_ORF9b.fasta',
 'data/v6/aligned_protein_S.fasta']

In [3]:
### Finding the reference sequence !!!!
for aa_file in ['data/v6/aligned_protein_ORF1a.fasta']: # amino_acid_files:
    protein = aa_file.split('_')[-1].split('.')[0]
    with open(aa_file, "rt") as handle:
        records = list(SeqIO.parse(handle, "fasta"))

    # getting the reference sequence !!!!!!!!!!!!!
    for ind, r in enumerate(records): 
        if r.id == 'Wuhan/IPBCAMS-WH-01/2019':
            ref_seq_ind = ind
            
    ref_seq = str(records[ref_seq_ind].seq)[:-1]
        

In [4]:
len(ref_seq)

4400

In [5]:
ref_seq;

In [6]:
ref_seq = ref_seq + 'NGFAV' # why is the very end of this not on my sequence? 

In [7]:
len(ref_seq)

4405

In [16]:
ref_seq;

In [17]:
# the very ending of this goes FLN. 1ab superset does not have the 'GFAV' portion. 
# this is also where the frame shift happens. if my protein is shorter though then that is fine. 

In [18]:
# parsing the ORF1a

# NB see if these txt files have been updated at all with new ORFS!!! 
# Copied and pasted text from the ORF online (see txt file for formatting)

with open('data/ORF1a.txt', 'r') as f: 
    lines = f.readlines()

In [19]:
to_df = []
for ind, l in enumerate(lines):
    if 'CHAIN' in l: 
        
        start, end = l.split('     ')[-1].strip().split('..')
        start, end = int(start), int(end)
        protein_name = lines[ind+1].split('"')[1]
        
        if 'Replicase polyprotein 1' in protein_name:
            continue
        
        to_df.append( [ start, end, protein_name ])
to_df        

[[1, 180, 'Non-structural protein 1'],
 [181, 818, 'Non-structural protein 2'],
 [819, 2763, 'Non-structural protein 3'],
 [2764, 3263, 'Non-structural protein 4'],
 [3264, 3569, '3C-like proteinase'],
 [3570, 3859, 'Non-structural protein 6'],
 [3860, 3942, 'Non-structural protein 7'],
 [3943, 4140, 'Non-structural protein 8'],
 [4141, 4253, 'Non-structural protein 9'],
 [4254, 4392, 'Non-structural protein 10'],
 [4393, 4405, 'Non-structural protein 11']]

In [20]:
orf1a = pd.DataFrame(to_df, columns=['Start', 'End', 'Sub_Protein'])
orf1a['Protein'] = 'ORF1a'
orf1a

,Start,End,Sub_Protein,Protein
0,1,180,Non-structural protein 1,ORF1a
1,181,818,Non-structural protein 2,ORF1a
2,819,2763,Non-structural protein 3,ORF1a
3,2764,3263,Non-structural protein 4,ORF1a
4,3264,3569,3C-like proteinase,ORF1a
5,3570,3859,Non-structural protein 6,ORF1a
6,3860,3942,Non-structural protein 7,ORF1a
7,3943,4140,Non-structural protein 8,ORF1a
8,4141,4253,Non-structural protein 9,ORF1a
9,4254,4392,Non-structural protein 10,ORF1a


# ORF1ab is a superset of ORF1a. 

In [21]:
for aa_file in ['data/v6/aligned_protein_ORF1b.fasta']: # amino_acid_files:
    protein = aa_file.split('_')[-1].split('.')[0]
    with open(aa_file, "rt") as handle:
        records = list(SeqIO.parse(handle, "fasta"))

    # getting the reference sequence
    for ind, r in enumerate(records): 
        if r.id == 'Wuhan/IPBCAMS-WH-01/2019':
            ref_seq_ind = ind
            
    ref_seq = str(records[ref_seq_ind].seq)[:-1]
        

In [22]:
len(ref_seq)

2695

In [23]:
# my sequences total length
4400+2695

7095

In [24]:
ref_seq;

In [25]:
with open('data/ORF1ab_FullSequence.txt', 'r') as f: 
    lines = f.readlines()
fullseq =''
for l in lines: 
    fullseq += l.strip().replace(' ', '')
fullseq;

In [27]:
ref_seq[:10]

'RVCGVSAARL'

In [26]:
# finding the start of 1b out of the 1ab superset.
start_of_1b = fullseq.index(ref_seq[:10]) # this is where 1b starts in 1ab. # as a zero indexed coordinate!!!
start_of_1b

4401

In [28]:
len(fullseq)

7096

In [29]:
len(fullseq[start_of_1b:])

2695

In [30]:
len(ref_seq)

2695

In [34]:
list(fullseq[start_of_1b:]) == ref_seq

False

In [37]:
print(ref_seq[:10])
print(fullseq[start_of_1b:start_of_1b+10])

RVCGVSAARL
RVCGVSAARL


In [48]:
mask = np.asarray(list(ref_seq)) != np.asarray(list(fullseq[start_of_1b:]))
len(mask)
mask;
mask.sum() #  = 1, so there is only one amino acid in which ref_seq and fullseq[start_of_1b:] differ

1

In [49]:
np.asarray(list(ref_seq))[mask]

array(['L'], dtype='<U1')

In [50]:
np.asarray(list(fullseq[start_of_1b:]))[mask]

array(['I'], dtype='<U1')

In [51]:
### one mutation here which is fine. this is the sequencing read error one that nextstrain accounts for. 

In [52]:
# parsing the ORF1a

with open('data/ORF1ab.txt', 'r') as f: 
    lines = f.readlines()

In [53]:
to_df = []
for ind, l in enumerate(lines):
    if 'CHAIN' in l: 
        
        start, end = l.split('     ')[-1].strip().split('..')
        start, end = int(start), int(end)
        protein_name = lines[ind+1].split('"')[1]
        
        if end < start_of_1b:
            continue
            
        if 'Replicase polyprotein 1' in protein_name:
            continue
        
        to_df.append( [ start-start_of_1b, end -start_of_1b, protein_name ])
to_df     

[[-8, 923, 'RNA-directed RNA polymerase'],
 [924, 1524, 'Helicase'],
 [1525, 2051, 'Guanine-N7 methyltransferase'],
 [2052, 2397, 'Uridylate-specific endoribonuclease'],
 [2398, 2695, "2'-O-methyltransferase"]]

In [54]:
orf1b = pd.DataFrame(to_df, columns=['Start', 'End', 'Sub_Protein'])
orf1b['Protein'] = 'ORF1b'
orf1b

,Start,End,Sub_Protein,Protein
0,-8,923,RNA-directed RNA polymerase,ORF1b
1,924,1524,Helicase,ORF1b
2,1525,2051,Guanine-N7 methyltransferase,ORF1b
3,2052,2397,Uridylate-specific endoribonuclease,ORF1b
4,2398,2695,2'-O-methyltransferase,ORF1b


In [55]:
# ensuring these are now aligned. 

In [56]:
fullseq[923+start_of_1b-5:923+start_of_1b+5]

'HTVLQAVGAC'

In [57]:
ref_seq[923-5:923+5]

'HTVLQAVGAC'

In [58]:
# the ends are the AA right before the next cleavage site. 

df= orf1a.append(orf1b)
df

,Start,End,Sub_Protein,Protein
0,1,180,Non-structural protein 1,ORF1a
1,181,818,Non-structural protein 2,ORF1a
2,819,2763,Non-structural protein 3,ORF1a
3,2764,3263,Non-structural protein 4,ORF1a
4,3264,3569,3C-like proteinase,ORF1a
5,3570,3859,Non-structural protein 6,ORF1a
6,3860,3942,Non-structural protein 7,ORF1a
7,3943,4140,Non-structural protein 8,ORF1a
8,4141,4253,Non-structural protein 9,ORF1a
9,4254,4392,Non-structural protein 10,ORF1a


In [59]:
# uniprot is 1 indexed. 
df['Start'] = df['Start']-1
df['End'] = df['End']-1

In [60]:
df.head()

,Start,End,Sub_Protein,Protein
0,0,179,Non-structural protein 1,ORF1a
1,180,817,Non-structural protein 2,ORF1a
2,818,2762,Non-structural protein 3,ORF1a
3,2763,3262,Non-structural protein 4,ORF1a
4,3263,3568,3C-like proteinase,ORF1a


furin cleavage in S1 and S2 https://link.springer.com/article/10.1007/s12250-020-00212-7
This furin cleavage site was located between the residues 682 and 685. RRAR is the sequence

In [61]:
for aa_file in ['data/v6/aligned_protein_S.fasta']: # amino_acid_files:
    protein = aa_file.split('_')[-1].split('.')[0]
    with open(aa_file, "rt") as handle:
        records = list(SeqIO.parse(handle, "fasta"))

    # getting the reference sequence
    for ind, r in enumerate(records): 
        if r.id == 'Wuhan/IPBCAMS-WH-01/2019':
            ref_seq_ind = ind
            
    ref_seq = str(records[ref_seq_ind].seq)[:-1]

In [62]:
len(ref_seq)

1273

In [63]:
ref_seq.index('RRAR')

681

In [64]:
ref_seq[681:686] #RRAR|S is the cleavage site. 

'RRARS'

In [65]:
ref_seq[685]

'S'

In [66]:
to_df = [[0, 684, 'S1', 'S'  ], [685, len(ref_seq), 'S2', 'S' ]]
S = pd.DataFrame(to_df, columns=['Start', 'End', 'Sub_Protein', 'Protein'])

df = df.append( S  )

In [67]:
df.tail()

,Start,End,Sub_Protein,Protein
2,1524,2050,Guanine-N7 methyltransferase,ORF1b
3,2051,2396,Uridylate-specific endoribonuclease,ORF1b
4,2397,2694,2'-O-methyltransferase,ORF1b
0,0,684,S1,S
1,685,1273,S2,S


In [68]:
df.to_csv('data/processed/ProteinCleavageSites.csv', index=False)